In [83]:
import numpy as np   #importing numpy library
import pandas as pd  #for data handling

In [84]:
train_df=pd.read_csv('train.csv')    #loading train data set 
test_df=pd.read_csv('test.csv')      #loading test data set
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [85]:
missing_values_train=train_df.columns[train_df.isnull().any()] #Null values containing columns in train data set
print(missing_values_train)
missing_values_test=test_df.columns[test_df.isnull().any()]    #Null values containing columns in test data set
print(missing_values_test)

Index(['Age', 'Cabin', 'Embarked'], dtype='object')
Index(['Age', 'Fare', 'Cabin'], dtype='object')


In [86]:
train_df=train_df.drop(['Cabin', 'PassengerId','Name','Ticket'], axis=1)  #Removing Cabin and Passenger Id columns as they are not related with Survival rate
test_df=test_df.drop(['Cabin', 'PassengerId','Name','Ticket'], axis=1)    #Removing Cabin and Passenger Id columns as they are not related with Survival rate
combine=[train_df,test_df]

In [87]:
#Filling NaN values in Embarked with mode i.e,the most previously occured value
freq_port = train_df.Embarked.dropna().mode()[0]                #finding mode value
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

    
#Converting Embarked to numerical feature
for dataset in combine:
    dataset['Embarked']=dataset['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)



#Converting age(categorical feature) to numerical feature
for dataset in combine:
    dataset['Sex']=dataset['Sex'].map({'female':1,'male':0}).astype(int)

In [88]:
#As age is reated with Survival so it has to be completed
#Guess Age values using median values for Age across sets of Pclass and Gender feature combinations for better results
guess_ages = np.zeros((2,3))
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)
combine=[train_df,test_df]

In [89]:
#Creating a feature family to replace SibSp and Parch as they contribute to Survival Rate
for dataset in combine:
    dataset['FamilySize']=dataset['SibSp']+dataset['Parch']
    

#Now remove SibSp and Parch
train_df=train_df.drop(['SibSp','Parch'], axis=1)
test_df=test_df.drop(['SibSp','Parch'], axis=1)

    
#Creating age bands
for dataset in combine:
    dataset.loc[dataset['Age']<= 16, 'Age']= 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age']= 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age']= 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age']= 3
    dataset.loc[dataset['Age'] > 64, 'Age']

In [98]:
#Filling Null Values for Fare in test data set
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)


combine=[train_df,test_df]

#Creating fare bands
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare']=dataset['Fare'].astype(int)

In [99]:
X_train = train_df.drop('Survived', axis=1)
Y_train = train_df['Survived']
X_test  = test_df.copy()

In [101]:
from sklearn.linear_model import LogisticRegression# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

c:\python2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


81.14